In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import time
from math import sqrt
import itertools


def extract_features(index, dataset): # function that takes list of feature names and returns those features only from dataset
    return dataset[index]

In [ ]:
filename = input("Type in the name of the file to test: ")
print("1) Forward Selection \n2) Backward Elimination") # User input for the algorithm
algo = int(input("Type the number of the algorithm you want to run:"))
data = open(filename).read() # reading dataset
data = data.split("\n") # spliting whole data with new lines

X = []
y = []

for i in data:
    if len(i) > 1: 
        i = i.strip() # removing extra spaces
        line = i.split("  ") # spliting each feature within single data point
        y.append(float(line[0])) # storing the target column
        single_instance = [] 
        for j in line[1:]:
            single_instance.append(float(j)) # stroing all features of single instance (data point)
        X.append(single_instance) 
        
df = pd.DataFrame(X) # converting the list into a pandas dataframe
arr = df.to_numpy()
df.columns = range(1, len(X[0]) + 1) # naming the column with feature names
# CS170_Small_Data__96.txt

In [ ]:
def knn(data, y, k = 5):
    neighbours = []
    predictions = []
    
    for index in range(len(data)): # iterating over all the datapoints
        neighbours = []
        datapoint = data.iloc[index]  # taking single datapoint
        edf = pow((datapoint - data), 2) # taking difference of the datapoint with rest of the datapoints in the dataset
        sumdf = edf.sum(axis = 1) # taking sum of the rows we got 
        sumdf = sumdf **(1/2) # euclidean # taking square root of the sum of the rows
        # so far we have calculated the scores 
        sortedseries = sumdf.sort_values(ascending=True)  # sorted scores
        n = list(sortedseries.index)[1: k + 1] # taking k nearest neighbours
        for i in n: 
            neighbours.append(y[i])  
        res = max(set(neighbours), key = neighbours.count) # taking majority of the classes
        predictions.append(res) # appending the prediction (majority) to the list
    return predictions

In [ ]:
start = time.time()
final_features = []
best = 0.0

visited = []


graph_x = []
graph_y = []

best_feature = None
alltimebest = []
if algo == 1: # forward selection
    for i in range(1, len(X[0]) + 2): # iterating over all the features
        level = []
        for subset in itertools.combinations(range(1, len(X[0]) + 1), i):
            features = list(subset) # adding new feature
            if best_feature:
                if not (set(best_feature).issubset(set(features))):
                    continue      
            if set(features) in visited:
                continue
            visited.append(set(features))
            
            edf = extract_features(features, df) # extracting only features that are in features list
            predictions = knn(edf, y) # initializing the KNN model
            score = accuracy_score(y, predictions)
            
            print("Features: ", set(features), "Accuracy: ", round(score*100,2))
            level.append((score, features))
            if score > best: # if the evaluated score is greater than previous best score
                best = score # update best score 
                final_features = features
                graph_x.append(features)
                graph_y.append(score)
        if len(level) != 0:
            best_feature = max(level)
            alltimebest.append(best_feature)
            print("Best Feature is: ", best_feature[1], "with accuracy:", round(best_feature[0]*100,2), "\n")
            best_feature = best_feature[1]
else:
    visited = []
    features = list(range(1, len(X[0]) + 1))
    edf = extract_features(features, df) # extracting the feature
    predictions = knn(edf, y) # initializing the KNN model
    best = accuracy_score(y, predictions)
    print("Features: ", set(features), "Accuracy: ", round(best*100,2)) 
    for i in reversed(range(1, len(X[0]))): # iterating the loop in reverse order
        temp = features.copy() # copying all the values
        temp.pop((i)) # eliminating the last feature
        if set(temp) in visited:
            continue
        visited.append(set(temp))
        edf = extract_features(temp, df) # extracting the feature
        predictions = knn(edf, y) # initializing the KNN model
        score = accuracy_score(y, predictions)
        print("Features: ", set(temp), "Accuracy: ", round(score*100,2))
        if score > best: # of score 
            best = score
            features = temp
            final_features = features
            graph_x.append(features)
            graph_y.append(score)

# # "CS170_Small_Data__96.txt"
# final_features
end = time.time()
graph_x = [str(x) for x in graph_x]
timetaken = str((end - start))[0:4]
print("Final Features: ", final_features)
print("Algorithm took: ", timetaken, "seconds")

print("\n\n")

import matplotlib.pyplot as plt

plt.plot(graph_x, graph_y)
plt.xlabel('x - Features')
plt.ylabel('y - Accuracy')
plt.show()